In [ ]:
!pip install torch transformers
!pip install -U datasets
!git clone https://github.com/davidjurgens/citation-function

In [1]:
import os
with open('api_openrouter.txt', 'r') as f:
  API=f.read()


In [2]:
from datasets import load_dataset
ds = load_dataset("hrithikpiyush/acl-arc")
ds.set_format("pandas")
train=ds["train"][:]
validation=ds["validation"][:]
test=ds["test"][:]

/home/mcherif/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
for text,intent in zip(test['text'],test['intent']):
    print('You are an expert in scientometry, your role is to classify the intent of a citation in a scientific paper. The possible intents are: Background, Uses, Extends, Motivation, CompareOrContrast, Future, Other. Given the text of the citation, classify the intent of the citation into one of these categories.Think as much as you want, but answer only with the associated number:'
            + 'From the following list: 0:BACKGROUND \n 1:USES, 2:COMPARES OR CONTRASTS, 3:MOTIVATION, 4:CONTINUATION, 5:FUTURE\n')
    print(text+'\n\n')

You are an expert in scientometry, your role is to classify the intent of a citation in a scientific paper. The possible intents are: Background, Uses, Extends, Motivation, CompareOrContrast, Future, Other. Given the text of the citation, classify the intent of the citation into one of these categories.Think as much as you want, but answer only with the associated number:From the following list: 0:BACKGROUND 
 1:USES, 2:COMPARES OR CONTRASTS, 3:MOTIVATION, 4:CONTINUATION, 5:FUTURE

2 Resnik ( 1995 ) reported a correlation of r = .9026.10 The results are not directly comparable , because he only used noun-noun pairs , words instead of concepts , a much smaller dataset , and measured semantic similarity instead of semantic relatedness .


You are an expert in scientometry, your role is to classify the intent of a citation in a scientific paper. The possible intents are: Background, Uses, Extends, Motivation, CompareOrContrast, Future, Other. Given the text of the citation, classify the i

In [ ]:
dico_rez={
    0:[]
    1:[]
    2:[]
    3:[]
    4:[]
    5:[]
}

NameError: name 'dico_rez' is not defined

In [ ]:
import requests
import json
import os

les_rez=[]
dico_rez={0:[],1:[],2:[],3:[],4:[],5:[]}

url = "https://openrouter.ai/api/v1/chat/completions"


headers = {
  "Authorization": f"Bearer {API}",
  "Content-Type": "application/json"
}




for text,intent in zip(test['text'],test['intent']):
  PROMPT=('You are an expert in scientometry, your role is to classify the intent of a citation in a scientific paper. The possible intents are: Background, Uses, Extends, Motivation, CompareOrContrast, Future, Other. Given the text of the citation, classify the intent of the citation into one of these categories.Think as much as you want, but answer only with the associated number:'
          + 'From the following list: 0:BACKGROUND \n 1:USES, 2:COMPARES OR CONTRASTS, 3:MOTIVATION, 4:CONTINUATION, 5:FUTURE\n')
  print(text+'\n\n')
  try:
    payload = {
        "model": "deepseek/deepseek-r1-0528:free",
      "messages": [
        {
          "role": "user",
          "content": PROMPT
        }
      ]
        ,
    #   "provider": {
    #     "allow_fallbacks": True,
    #     "quantizations": [
    #       "fp8"
    #     ]
    # },
        "reasoning": {

      "enabled": True  # Use high reasoning effort

  }
    }
    response = requests.post(url, headers=headers, json=payload)
    les_rez.append(response.json())
    dico_rez[intent].append(response.json())
  except:
    print("error")
    continue
  print(response.json())

NOUVELLE CLASSE: 0
0
{'id': 'gen-1758487456-SwfESyvaD0piczSzsA2C', 'provider': 'DeepInfra', 'model': 'deepseek/deepseek-chat-v3.1:free', 'object': 'chat.completion', 'created': 1758487456, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': 'The study of Renaissance art is fundamentally rooted in the socio-political context of the Italian city-states, where patronage systems fueled artistic innovation (@@CITATION).\n\nGiorgio Vasari\'s seminal text, *The Lives of the Artists*, established a critical framework for understanding the period\'s artistic development and remains a primary source for art historians (@@CITATION).\n\nThe intellectual shift towards humanism, which placed renewed emphasis on classical antiquity and the individual, provided the philosophical foundation for Renaissance artistic expression (@@CITATION).\n\nThe development of linear perspective, systematically articulated by F

In [8]:
for i in range(0,6):
    for elt in dico_rez[i]:
        with open(f'DATASET3/results_openrouter_intent_{i}.txt', 'a',encoding='utf-8') as f:
            f.write(elt['choices'][0]['message']['content'] + '\n\n')